# **YoloV3(TensorFlow) 모델 실습**
- TensorFlow 기반의 YoloV3 모델을 활용한 객체 탐지(Object Detection) 실습
- VOC 데이터를 사용해 실습을 진행하며, 이미지에서 물체를 탐지하고 결과를 시각화 방법

## **1. 사전 환경 세팅**
- 필요한 라이브러리 설치 및 불러오기
- 모델 파일 및 관련 설정 파일 준비

### **1-1. TensorFlow 정상 설치 확인**
- TensorFlow는 딥러닝 모델을 구축 및 학습시키는 데 주로 사용되는 프레임워크
- 설치 확인을 통해 실행 환경이 올바르게 설정되었는지 점검

In [ ]:
# Conda 환경 생성 및 활성화
# 새로운 Conda 환경(day14-1)을 생성하고 Python 3.9 버전 설치

# day14-1이라는 이름의 Conda 환경 생성
# conda create -n day14-1 python=3.9

# 생성한 day14-1 환경 활성화
# conda activate day14-1

# 필요한 패키지 설치 및 Jupyter Notebook 설정
# requirements.txt 파일을 통해 필요한 패키지 설치
# pip install -r requirements.txt

# Jupyter Notebook 설치
# pip install jupyter notebook

# Jupyter Notebook에서 새로운 Conda 커널 설정
# python -m ipykernel install --user --name day14-1 --display-name day14-1

In [ ]:
# TensorFlow 라이브러리 불러오기
import tensorflow as tf

# TensorFlow 버전 확인
tf.__version__  # TensorFlow 버전 출력

### **1-2. Darknet weight 파일로 변환 처리**
- YOLOv3 모델의 Darknet 프레임워크에서 사용되던 가중치 파일을 TensorFlow에서 사용할 수 있도록 변환
- Darknet은 YOLO 모델의 초기 구현에 사용된 프레임워크에 해당하며, 모델의 학습된 정보를 유지하면서 TensorFlow에서 활용할 수 있도록 가중치 파일을 전처리

In [ ]:
# Darknet 가중치 파일을 TensorFlow 포맷으로 변환
# YOLOv3 가중치 파일 변환
# - 'convert.py': Darknet 가중치를 TensorFlow로 변환하는 스크립트
# - '--weights ./data/yolov3.weights': Darknet 가중치 파일 경로를 지정
# - '--output ./checkpoints/yolov3.tf': 변환된 TensorFlow 포맷의 출력 파일 경로
!python convert.py --weights ./data/yolov3.weights --output ./checkpoints/yolov3.tf

# YOLOv3-tiny 가중치 파일 변환
# - YOLOv3-tiny는 경량화된 버전의 YOLOv3로, 더 빠른 처리가 가능하지만 정확도는 다소 낮을 수 있음
# - '--tiny': 변환 대상이 YOLOv3-tiny임을 명시
# - 다른 인자는 YOLOv3와 동일
!python convert.py --weights ./data/yolov3-tiny.weights --output ./checkpoints/yolov3-tiny.tf --tiny

## **2. Detector 정의**
- 이 섹션에서는 YOLOv3 객체 탐지 모델을 정의합니다.
- 변환된 TensorFlow 포맷의 YOLOv3 모델을 로드하고 초기화합니다.
- Detector는 이미지를 입력받아 탐지 결과를 반환하는 주요 구성 요소입니다.

In [ ]:
# 필요한 라이브러리 및 사용자 정의 모듈 불러오기

import sys # Python 인터프리터와 상호작용하기 위한 모듈
from absl import app, logging, flags # app: 명령줄 인터페이스를 간단히 처리, logging: 로그 메시지를 출력, flags: 명령줄 플래그를 정의하고 처리
from absl.flags import FLAGS
import time # 코드 실행 시간을 측정하거나 지연을 구현
import cv2 # cv2: 컴퓨터 비전 작업(이미지 읽기, 처리 등)을 위한 모듈
import numpy as np # np: 배열 및 수학 연산을 처리하기 위한 라이브러리
import tensorflow as tf # tf: YOLOv3 모델 구현에 사용
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
) # YoloV3, YoloV3Tiny: YOLOv3 모델과 경량 YOLOv3-tiny 모델 정의
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset # transform_images: 이미지 전처리 함수, load_tfrecord_dataset: TFRecord 데이터셋 로드
from yolov3_tf2.utils import draw_outputs # draw_outputs: 탐지 결과를 이미지 위에 시각화

In [ ]:
# absl 라이브러리를 사용해 명령줄 플래그(옵션)를 정의하고, 이를 통해 YOLOv3 모델의 동작을 유연하게 제어할 수 있도록 설정 제공

# 클래스 이름 파일 경로 설정
# - COCO 데이터셋의 클래스 이름이 포함된 파일 경로를 지정
flags.DEFINE_string('classes', './data/coco.names', 'path to classes file')

# YOLOv3 모델 가중치 파일 경로 설정
# - 변환된 YOLOv3 TensorFlow 가중치 파일 경로를 지정
flags.DEFINE_string('weights', './checkpoints/yolov3.tf', 'path to weights file')

# YOLOv3 모델 유형 선택
# - True로 설정하면 YOLOv3-tiny 모델 사용, 기본값은 False
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')

# 입력 이미지 크기 설정
# - YOLOv3 모델은 입력 이미지를 416x416 크기로 리사이즈하여 처리
flags.DEFINE_integer('size', 416, 'resize images to')

# 입력 이미지 파일 경로 설정
# - 분석할 입력 이미지 파일의 경로를 지정
flags.DEFINE_string('image', './data/girl.png', 'path to input image')

# TFRecord 데이터셋 경로 설정 (선택 사항)
# - TFRecord 형식의 데이터셋 경로를 지정, 기본값은 None
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')

# 출력 이미지 파일 경로 설정
# - 탐지 결과가 저장될 이미지 파일 경로를 지정
flags.DEFINE_string('output', './output.jpg', 'path to output image')

# 모델 클래스 수 설정
# - YOLOv3 모델의 클래스 개수를 지정 (기본값: 80)
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')

# 앱 초기화 및 명령줄 플래그 파싱
# - 앱 실행 준비 및 플래그 값 파싱
app._run_init(['yolov3'], app.parse_flags_with_usage)

In [ ]:
# GPU 설정을 통해 사용 가능한 GPU 장치를 확인하고, GPU 메모리 활성화

# 사용 가능한 GPU 장치 확인
# 'tf.config.experimental.list_physical_devices('GPU')'는 시스템에서 사용 가능한 GPU 장치 목록 반환
# 이를 통해 GPU가 올바르게 감지되었는지 확인
physical_devices = tf.config.experimental.list_physical_devices('GPU')

# GPU 메모리 동적 할당 설정
# 'set_memory_growth': GPU 메모리를 필요한 만큼만 동적으로 할당
# 일반적으로 TensorFlow는 실행 시 모든 GPU 메모리를 예약하지만, 동적 설정을 통해 효율적인 메모리 사용이 가능
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## **3. pre-trained weight를 활용한 detection 테스트**
- 이 섹션에서는 사전 학습된 YOLOv3 가중치를 사용하여 객체 탐지 작업을 수행합니다.
- 사전 학습된 가중치는 COCO 데이터셋에서 훈련된 YOLOv3 모델의 학습 정보를 포함합니다.
- 이를 통해 모델의 성능을 확인하고 탐지 결과를 시각화합니다.

In [ ]:
# 테스트할 이미지 입력 경로 설정
# FLAGS.image를 통해 탐지 테스트에 사용할 입력 이미지의 경로 지정
# 'data/meme.jpg'은 샘플 이미지 경로
FLAGS.image = 'data/street_out.jpg' # meme2, girl, street_out, street 로도 테스트 

# YOLOv3 기본 또는 tiny 모델 파이프라인 선택
# FLAGS.tiny: True이면 YOLOv3-tiny 모델을 사용, False이면 일반 YOLOv3 모델을 사용
# classes: 모델이 탐지할 클래스 수를 설정 (기본 80 클래스)
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)  # 경량 YOLOv3-tiny 모델 생성
else:
    yolo = YoloV3(classes=FLAGS.num_classes)  # YOLOv3 모델 생성

# 모델 weight 파일 로드
# FLAGS.weights 경로에 저장된 사전 학습된 가중치를 모델에 로드
# expect_partial(): 일부 레이어가 맞지 않을 경우에도 로드 허용
yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')  # 가중치 로드 완료 메시지

# 클래스 이름 로드
# FLAGS.classes 파일에 저장된 클래스 이름을 리스트로 읽어오기
# c.strip(): 클래스 이름에서 불필요한 공백 제거
class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')  # 클래스 로드 완료 메시지

# 입력 이미지 로드
# FLAGS.image 경로에서 이미지를 바이너리 모드로 읽어와 디코딩
# channels=3: 이미지를 RGB 채널로 디코딩
img_raw = tf.image.decode_image(
    open(FLAGS.image, 'rb').read(), channels=3)

# 이미지 전처리
# 차원 확장: 모델에 입력하기 위해 배치 차원을 추가
img = tf.expand_dims(img_raw, 0)
# 리사이즈 및 정규화: 이미지를 YOLOv3의 입력 크기(FLAGS.size)로 리사이즈하고 픽셀 값을 정규화
img = transform_images(img, FLAGS.size)

In [ ]:
# 모델 예측 수행

# 시작 시간 기록
# 'time.time()'을 사용해 객체 탐지 수행 전의 시간 기록
t1 = time.time()

# 객체 탐지 수행
# yolo(img): YOLOv3 모델이 입력 이미지를 분석하고 객체 탐지
# 반환값:
# - boxes: 탐지된 객체의 경계 상자 좌표
# - scores: 각 객체의 신뢰도 점수
# - classes: 탐지된 객체의 클래스 번호
# - nums: 탐지된 객체의 총 개수
boxes, scores, classes, nums = yolo(img)

# 종료 시간 기록
# 탐지 작업 완료 시점 기록
t2 = time.time()

# 탐지 시간 출력
# 탐지 수행 시간(t2 - t1) 출력
logging.info('time: {}'.format(t2 - t1))

# 탐지된 객체 정보 출력
# 'detections:' 메시지 출력
# - 탐지된 객체 정보를 출력하기 전 시작 메시지 로그
logging.info('detections:')

# 탐지된 객체를 하나씩 출력
# - nums[0]: 탐지된 객체의 총 개수
# - 클래스 이름, 신뢰도 점수, 경계 상자 좌표를 순서대로 로그에 출력
for i in range(nums[0]):
    logging.info('\t{}, {}, {}'.format(
        class_names[int(classes[0][i])],  # 클래스 이름
        np.array(scores[0][i]),          # 신뢰도 점수
        np.array(boxes[0][i])            # 경계 상자 좌표
    ))

In [ ]:
# 탐지 결과 이미지로 표시

# 이미지 색상 변환
# - OpenCV는 BGR 형식을 사용하므로, TensorFlow로 불러온 RGB 이미지를 BGR로 변환
# - img_raw.numpy(): TensorFlow 텐서를 NumPy 배열로 변환
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)

# 탐지 결과 시각화
# - draw_outputs(): 탐지된 경계 상자와 클래스 이름, 신뢰도 점수를 이미지 위에 표시
# - img: 원본 이미지
# - (boxes, scores, classes, nums): 탐지된 객체 정보
# - class_names: 클래스 이름 리스트
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

# Jupyter Notebook에서 이미지 표시
# IPython.display 모듈 불러오기
# - Jupyter Notebook에서 이미지를 표시하기 위한 모듈
from IPython.display import Image, display

# 이미지 디스플레이
# - cv2.imencode(): 이미지를 특정 형식(예: JPEG)으로 인코딩
# - bytes(): 인코딩된 이미지를 바이트로 변환
# - width=800: 표시할 이미지의 너비를 설정
display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))

## **4. 신규 학습 진행**
- 새로운 데이터셋을 사용하여 YOLOv3 모델 학습
- 사전 학습된 가중치를 활용하거나 초기화된 모델로 시작 가능
- 학습된 모델은 사용자 정의 데이터셋에서 물체를 탐지하도록 최적화

### **4-1. 데이터 전처리**
- 학습에 사용할 데이터셋을 모델 입력 형식에 맞게 변환
- 이미지 크기 조정, 정규화, TFRecord 로드 등 전처리 작업에 해당

In [ ]:
!pwd

In [ ]:
# 데이터셋을 생성할 위치(/Day-14/yolov3)로 이동
# %cd /{}/Day-14/yolov3

In [ ]:
# raw 데이터(tar) 압축 해제

# 데이터셋 다운로드 (S3 드라이브에서 voc2009_raw.tar 파일 다운로드, Day-14/yolov3/data/ 위치로 이동)
# - wget 명령으로 Pascal VOC 2009 데이터셋을 다운로드
# - '-O ./data/voc2009_raw.tar': 다운로드한 파일을 지정된 경로에 저장
# !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2009/VOCtrainval_11-May-2009.tar -O ./data/voc2009_raw.tar

# 데이터셋 디렉토리 생성
# - 'mkdir -p': 지정된 경로에 디렉토리를 생성 (이미 존재하면 무시)
!mkdir -p ./data/voc2009_raw

# tar 파일 압축 해제
# - 'tar -xf': 지정된 tar 파일을 해제
# - '-C ./data/voc2009_raw': 압축 해제된 데이터를 지정된 디렉토리로 저장
!tar -xf ./data/voc2009_raw.tar -C ./data/voc2009_raw

In [ ]:
# YOLOv3 학습을 위한 voc2012 학습 데이터 전처리 수행

# 1. voc2012.py 스크립트를 실행하여 데이터셋을 변환
# - '--data_dir': 원본 데이터셋 경로를 지정
# - '--split': 사용할 데이터 분할을 지정 (train은 학습용 데이터)
# - '--output_file': 변환된 TFRecord 파일의 저장 경로
!python3 tools/voc2012.py \
  --data_dir './data/voc2009_raw/VOCdevkit/VOC2009' \
  --split train \
  --output_file ./data/voc_train.tfrecord

# YOLOv3 학습을 위한 voc2012 검증 데이터 전처리 수행

# 1. voc2012.py 스크립트를 실행하여 검증용 데이터를 변환
# - '--split val': 검증용 데이터를 지정
# - 결과는 './data/voc_val.tfrecord'에 저장
!python3 tools/voc2012.py \
  --data_dir './data/voc2009_raw/VOCdevkit/VOC2009' \
  --split val \
  --output_file ./data/voc_val.tfrecord

In [ ]:
!nvcc -V

In [ ]:
# 학습 파라미터 설정 및 학습 시작

# 1. 학습 스크립트 실행
# - 'train.py' 스크립트를 실행하여 YOLOv3 모델 학습 시작

# 2. 주요 파라미터
# - '--dataset': 학습 데이터셋(TFRecord 파일) 경로
# - '--val_dataset': 검증 데이터셋(TFRecord 파일) 경로
# - '--classes': 클래스 이름 파일 경로
# - '--num_classes': 학습할 데이터셋의 클래스 수 (Pascal VOC는 20개 클래스)
# - '--mode fit': Keras 모델 학습 모드 지정
# - '--transfer darknet': Darknet 가중치를 전이 학습으로 사용
# - '--batch_size': 학습에 사용할 배치 크기
# - '--epochs': 학습 반복 횟수
# - '--weights': 초기 가중치 파일 경로
# - '--weights_num_classes': 초기 가중치에 포함된 클래스 수 (COCO 데이터셋은 80개 클래스)

!python3 train.py \
	--dataset ./data/voc_train.tfrecord \
	--val_dataset ./data/voc_val.tfrecord \
	--classes ./data/voc2012.names \
	--num_classes 20 \
	--mode fit --transfer darknet \
	--batch_size 8 \
	--epochs 3 \
	--weights ./checkpoints/yolov3.tf \
	--weights_num_classes 80

## **5. 신규 학습 weight를 활용한 추론**
- 이 섹션에서는 새롭게 학습된 YOLOv3 가중치를 사용하여 객체 탐지를 수행합니다.
- 학습된 모델의 성능을 검증하고 탐지 결과를 시각화합니다.

In [ ]:
# 새로 학습된 모델을 활용한 추론 준비

# 모델이 인식할 클래스 수 설정
# - Pascal VOC 데이터셋은 총 20개의 클래스로 구성
FLAGS.num_classes = 20

# 클래스 이름 파일 경로 설정
# - 'data/voc2012.names' 파일 내 20개 클래스 이름 정의
FLAGS.classes = 'data/voc2012.names'

# 학습된 가중치 파일 경로 설정
# - 'checkpoints/yolov3_train_3.tf' 파일은 3 에포크 동안 학습된 YOLOv3 모델의 가중치를 저장한 파일
FLAGS.weights = 'checkpoints/yolov3_train_3.tf'

# 추론에 사용할 입력 이미지 경로 설정
# - 'data/meme.jpg'는 추론 테스트에 사용될 입력 이미지 파일
FLAGS.image = 'data/meme.jpg'

In [ ]:
# 낮은 임계값 설정 및 추론 진행

# IoU 임계값 설정
# - IoU(Intersection over Union): 예측된 바운딩 박스와 실제 바운딩 박스 간의 겹치는 비율
# - 낮은 임계값(0.1)을 설정하여 더 많은 바운딩 박스를 탐지
FLAGS.yolo_iou_threshold = 0.1

# 점수 임계값 설정
# - 예측된 바운딩 박스의 신뢰도 점수 임계값
# - 낮은 임계값(0.1)을 설정하여 더 많은 바운딩 박스를 표시
FLAGS.yolo_score_threshold = 0.1

In [ ]:
# 모델을 YOLOv3 Tiny 또는 기본 YOLOv3로 설정

# 1. 모델 초기화
# - FLAGS.tiny가 True이면 YOLOv3 Tiny 모델을 사용
# - False이면 YOLOv3 기본 모델을 사용
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)  # YOLOv3 Tiny 모델 초기화
else:
    yolo = YoloV3(classes=FLAGS.num_classes)  # YOLOv3 기본 모델 초기화

# 학습된 가중치 로드
# - FLAGS.weights 경로에서 학습된 가중치를 모델에 로드
# - expect_partial(): 일부 레이어가 맞지 않을 경우에도 로드 허용
yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')  # 가중치 로드 완료 메시지

# 클래스 이름을 로드
# - FLAGS.classes 경로에 정의된 클래스 이름을 리스트로 읽어옴
class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')  # 클래스 로드 완료 메시지

# 입력 이미지를 읽고 디코딩
# - FLAGS.image 경로에서 이미지를 바이너리 모드로 읽어와 디코딩
# - channels=3: 이미지를 RGB 채널로 디코딩
img_raw = tf.image.decode_image(
    open(FLAGS.image, 'rb').read(), channels=3
)

# 배치 차원을 추가하여 모델에 맞게 확장
# - YOLOv3 모델은 배치 입력을 요구하므로 차원을 확장
img = tf.expand_dims(img_raw, 0)

# 이미지를 모델 입력 크기로 변환
# - FLAGS.size에 설정된 크기 (예: 416x416)로 리사이즈 및 정규화
img = transform_images(img, FLAGS.size)

# 예측 수행

# 1. 예측 시작 시간 기록
t1 = time.time()

# 2. 모델을 사용하여 입력 이미지에서 객체 탐지
# - boxes: 탐지된 바운딩 박스 좌표
# - scores: 각 객체의 신뢰도 점수
# - classes: 탐지된 객체의 클래스 번호
# - nums: 탐지된 객체의 총 개수
boxes, scores, classes, nums = yolo(img)

# 3. 예측 종료 시간 기록
t2 = time.time()
logging.info('time: {}'.format(t2 - t1))  # 탐지 시간 출력

# 탐지 결과 로깅

# 1. 탐지된 객체 정보 출력 시작
logging.info('detections:')

# 2. 탐지된 객체의 클래스, 신뢰도 점수, 바운딩 박스 좌표를 하나씩 출력
for i in range(nums[0]):
    logging.info('\t{}, {}, {}'.format(
        class_names[int(classes[0][i])],  # 클래스 이름
        np.array(scores[0][i]),          # 신뢰도 점수
        np.array(boxes[0][i])            # 바운딩 박스 좌표
    ))

In [ ]:
# 이미지 색상 변환 (RGB에서 BGR로 변환하여 OpenCV에서 사용)
# - TensorFlow에서 로드된 이미지는 RGB 형식
# - OpenCV는 BGR 형식을 사용하므로 변환이 필요
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)

# 예측된 바운딩 박스와 클래스 이름을 이미지에 그림
# - draw_outputs(): 탐지된 객체 정보를 이미지 위에 시각화
# - img: 입력 이미지
# - (boxes, scores, classes, nums): 탐지 결과
# - class_names: 클래스 이름 리스트
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

# 결과 이미지를 파일로 저장
# - FLAGS.output: 저장할 파일 경로
cv2.imwrite(FLAGS.output, img)
logging.info('output saved to: {}'.format(FLAGS.output))  # 결과 저장 로그

# Jupyter Notebook에서 이미지를 표시
# - IPython.display 모듈을 사용하여 이미지 출력
from IPython.display import Image, display
# - cv2.imencode(): 이미지를 JPEG 형식으로 인코딩
# - bytes(): 인코딩된 이미지를 바이트로 변환
# - width=800: 출력 이미지의 너비 설정
display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))